## **ECG Diagnosis Code**

This code is based on the code developed here: https://doi.org/10.1038/s41467-020-15432-4

**Define Libraries**

In [21]:
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, Add, Flatten, Dense)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, ReduceLROnPlateau,
                                        CSVLogger, EarlyStopping)
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, f1_score, recall_score, confusion_matrix, accuracy_score
import numpy as np
import h5py
import math
import pandas as pd
from tensorflow.keras.utils import Sequence
import numpy as np
import os
import pickle

In [ ]:
cwd = os.getcwd()

**Load the data**

First Load in the test data

In [ ]:
#Load in test data
path_to_hdf5 = cwd + '\\data\\test\\ecg_tracings.hdf5'
dataset_name = 'tracings'
path_to_csv = cwd + '\\data\\test\\gold_standard.csv'

#Order is based on test set csv
abnormalities = ['1dAVb','RBBB', 'LBBB', 'SB',  'AF', 'ST']

labels_test = pd.read_csv(path_to_csv).values
f = h5py.File(path_to_hdf5, "r")
tracings_test = f[dataset_name][()]
f.close()

Investigation of the class probabilities in the test set

In [ ]:
def FindPercents(labels:np.ndarray, average: bool=True):
    bias = []
    for ii in range(labels.shape[-1]):
        bias.append(np.sum(labels[:,ii])/len(labels[:,ii]))
    
    if average:
        bias = np.mean(bias)
        return bias
    else:
        return np.array(bias)

test_per = FindPercents(labels_test)

**Saving Data Functions**

A function for saving data

In [ ]:
def SaveObject(object_to_save, save_path):
    '''Load path must have the file name with the .pickle extension'''
    pickle_out = open(save_path, 'wb')
    pickle.dump(object_to_save, pickle_out)
    pickle_out.close()

Function for loading data

In [ ]:
def LoadObject(load_path):
    infile = open(load_path, 'rb')
    Loaded_object = pickle.load(infile)
    infile.close()

    return Loaded_object

## Current Model

**Define the NN model**

In [ ]:
class ResidualUnit(object):
    def __init__(self, n_samples_out, n_filters_out, kernel_initializer='he_normal',
                 dropout_keep_prob=0.8, kernel_size=17, preactivation=True,
                 postactivation_bn=False, activation_function='relu'):
        self.n_samples_out = n_samples_out
        self.n_filters_out = n_filters_out
        self.kernel_initializer = kernel_initializer
        self.dropout_rate = 1 - dropout_keep_prob
        self.kernel_size = kernel_size
        self.preactivation = preactivation
        self.postactivation_bn = postactivation_bn
        self.activation_function = activation_function

    def _skip_connection(self, y, downsample, n_filters_in):
        """Implement skip connection."""
        # Deal with downsampling
        if downsample > 1:
            y = MaxPooling1D(downsample, strides=downsample, padding='same')(y)
        elif downsample == 1:
            y = y
        else:
            raise ValueError("Number of samples should always decrease.")
        # Deal with n_filters dimension increase
        if n_filters_in != self.n_filters_out:
            # This is one of the two alternatives presented in ResNet paper
            # Other option is to just fill the matrix with zeros.
            y = Conv1D(self.n_filters_out, 1, padding='same',
                       use_bias=False, kernel_initializer=self.kernel_initializer)(y)
        return y

    def _batch_norm_plus_activation(self, x):
        if self.postactivation_bn:
            x = Activation(self.activation_function)(x)
            x = BatchNormalization(center=False, scale=False)(x)
        else:
            x = BatchNormalization()(x)
            x = Activation(self.activation_function)(x)
        return x

    def __call__(self, inputs):
        """Residual unit."""
        x, y = inputs
        n_samples_in = y.shape[1]
        downsample = n_samples_in // self.n_samples_out
        n_filters_in = y.shape[2]
        y = self._skip_connection(y, downsample, n_filters_in)
        # 1st layer
        x = Conv1D(self.n_filters_out, self.kernel_size, padding='same',
                   use_bias=False, kernel_initializer=self.kernel_initializer)(x)
        x = self._batch_norm_plus_activation(x)
        if self.dropout_rate > 0:
            x = Dropout(self.dropout_rate)(x)

        # 2nd layer
        x = Conv1D(self.n_filters_out, self.kernel_size, strides=downsample,
                   padding='same', use_bias=False,
                   kernel_initializer=self.kernel_initializer)(x)
        if self.preactivation:
            x = Add()([x, y])  # Sum skip connection and main connection
            y = x
            x = self._batch_norm_plus_activation(x)
            if self.dropout_rate > 0:
                x = Dropout(self.dropout_rate)(x)
        else:
            x = BatchNormalization()(x)
            x = Add()([x, y])  # Sum skip connection and main connection
            x = Activation(self.activation_function)(x)
            if self.dropout_rate > 0:
                x = Dropout(self.dropout_rate)(x)
            y = x
        return [x, y]


def get_model(n_classes, last_layer='sigmoid'):
    kernel_size = 16
    kernel_initializer = 'he_normal'
    signal = Input(shape=(4096, 12), dtype=np.float32, name='signal')
    x = signal
    x = Conv1D(64, kernel_size, padding='same', use_bias=False,
               kernel_initializer=kernel_initializer)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x, y = ResidualUnit(1024, 128, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, x])
    x, y = ResidualUnit(256, 196, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x, y = ResidualUnit(64, 256, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x, _ = ResidualUnit(16, 320, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x = Flatten()(x)
    diagn = Dense(n_classes, activation=last_layer, kernel_initializer=kernel_initializer)(x)
    model = Model(signal, diagn)
    return model


**Load Parameters**

Loading the parameters for the model that were found in the paper. We will call this our first model

They trained 10 NN with different initializations. The choose the model to use based on the median micro average persion (mAP = 0.951). They had to choose the one right above the median since 10 is even so they can't take the median execution

In [ ]:
path_to_model = cwd + '\\model\\model.hdf5'

loss = 'binary_crossentropy'
lr = 0.001
batch_size = 64
opt = Adam(lr)

model_1 = load_model(path_to_model, compile=False)
model_1.compile(loss=loss, optimizer=Adam())

**Data Fromatting**

Here is the class for transforming the data into the proper format

In [ ]:
class ECGSequence(Sequence):
    @classmethod
    def get_train_and_val(cls, tracings: np.ndarray, labels: np.ndarray=None, batch_size=8, val_split=0.02):
        n_samples = tracings.shape[0]
        n_train = math.ceil(n_samples*(1-val_split))
        train_seq = cls(tracings, labels, batch_size, end_idx=n_train)
        valid_seq = cls(tracings, labels, batch_size, start_idx=n_train)
        return train_seq, valid_seq

    def __init__(self, tracings:np.ndarray, labels:np.ndarray=None, batch_size:int=8,\
        start_idx=0, end_idx=None):
        if labels is None:
            self.y = None
        else:
            self.y = labels
        # Get tracings
        self.x = tracings
        self.batch_size = batch_size
        if end_idx is None:
            end_idx = len(self.x)
        self.start_idx = start_idx
        self.end_idx = end_idx

    @property
    def n_classes(self):
        return self.y.shape[1]

    def __getitem__(self, idx):
        start = self.start_idx + idx * self.batch_size
        end = min(start + self.batch_size, self.end_idx)
        if self.y is None:
            return np.array(self.x[start:end, :, :])
        else:
            return np.array(self.x[start:end, :, :]), np.array(self.y[start:end])

    def __len__(self):
        return math.ceil((self.end_idx - self.start_idx) / self.batch_size)


**Training Function**

We will also train the model with the data accessible for better comparison with the simplified model. We will call this the second model

For sake of computational resources and time, the second model was only trained once instead of trained 10 times and then taking the model based on the median mAP

In [ ]:
class MyCNN:
    def __init__(self, loss, opt, verbose, save_path=None):
        # Optimization settings
        self.callbacks = [ReduceLROnPlateau(monitor='val_loss',
                            factor=0.1,
                            patience=7,
                            min_lr=lr / 100),
                            EarlyStopping(monitor='val_loss', 
                            patience=9,  # Patience should be larger than the one in ReduceLROnPlateau
                            min_delta=0.00001)]

        self.loss = loss
        self.optimizer = opt
        self.verbose = verbose
        
        # Save the BEST and LAST model
        if not save_path is None:
            self.callbacks += [ModelCheckpoint(save_path, save_best_only=True)]

    def train(self, train_seq, val_seq):
        self.model = get_model(train_seq.n_classes)
        self.model.compile(loss=self.loss, optimizer=self.optimizer)
        # Train neural network
        self.model.fit(train_seq,
            epochs=50,
            initial_epoch=0,  # If you are continuing a interrupted section change here
            callbacks=self.callbacks,
            validation_data=val_seq,
            verbose=self.verbose)

    def predict(self, test_seq):
        return self.model.predict(test_seq,  verbose=self.verbose)

## Simplified Models

Need to choose what model I want

Going to have to use something like random forest because I need a multi-label classifier, or I can use sklearn.multioutput.MultiOutputClassifier and use any classifier

I think all of the data for all 12 leads is the set of features for each sample

**Define Models**

In [ ]:
#TODO: Tune the hyperparameters
class RF_Model:
    def __init__(self, verbose = 1):
        self.model = RandomForestClassifier(verbose=verbose)

    def train(self, X: np.ndarray, y: np.ndarray):
        self.model.fit(X,y)

    def predict(self, X):
        return self.model.predict(X)

class LR_model:
    def __init__(self, verbose = 1):
        self.model = MultiOutputClassifier(LogisticRegression(verbose=verbose))

    def train(self, X: np.ndarray, y: np.ndarray):
        self.model.fit(X, y)

    def predict(self, X: np.ndarray):
        return self.model.predict(X)

**Data Fromatting**

PCA for the simplified models

In [ ]:
class PCA_Transform:
    def __init__(self, r:int):
        self.PCA_instance = PCA(n_components=r)

    def _processData(self, X: np.ndarray):
        self.preprocess = StandardScaler()
        self.preprocess.fit(X)

    def FitData(self, X: np.ndarray):
        self._processData(X)
        self.PCA_instance.fit(self.preprocess.transform(X))

    def TransformData(self, X_train: np.ndarray, X_test: np.ndarray):
        X_train = self.preprocess.transform(X_train)
        X_test = self.preprocess.transform(X_test)
        return self.PCA_instance.transform(X_train), self.PCA_instance.transform(X_test)

## K-Fold (FIX paragraph below)

K-fold procedure for validation of the models

They use a validation set of 2% so something to think about

They didn't round for the outputs, seems to be a threshold in which they consider it to occur

They used precision-recall curves for things, but in total found precision, recall, specificity and F1 score

**Process Outputs**

Function for processing the output probabilities from the CNNs

Need to find the optimal thresholds to maximize F1 score for the test set before running the processing

Will use these thresholds for the other CNN as well

In [54]:
def FindThresholds(y_pred: np.ndarray, y_true: np.ndarray):
    '''Optimize the thresholds based on the F1-score for the model 1 in the test set'''

    start = 0.05
    end = 0.8
    step = 0.001
    threshold_options = np.linspace(start,end,int((end-start)/step))
    optimal_thresholds = np.array([])

    for ii in range(y_pred.shape[1]):
        #use the np greater for a list of thresholds and then find max F1-score and index it back to the thresholds
        F1_scores = [f1_score(y_true=y_true[:,ii], y_pred=ProcessOutputs(y_pred[:,ii], threshold), zero_division=0) for threshold in threshold_options]
        optimal_thresholds = np.append(optimal_thresholds, threshold_options[np.argmax(F1_scores)])

    return optimal_thresholds


def ProcessOutputs(outputs: np.ndarray, thresholds: np.ndarray):

    threshold_check = np.array([np.greater_equal(sample, thresholds) for sample in outputs])
    threshold_outputs = threshold_check.astype(int)
    
    return threshold_outputs


#Find optimal thresholds with test set
test_seq = ECGSequence(tracings_test, labels_test, batch_size=32)
optimal_thresholds = FindThresholds(model_1.predict(test_seq), labels_test)

**Metrics Function**

Making a function to be able to call all of the metrics each fold

In [70]:
def Score_initator(filler):

        scores = dict()
        metrics = ['Accuracy','Precision', 'Recall', 'Specificity', 'F1']
        for ii, metric in enumerate(metrics):
                scores[metric] = dict()
                for zz, abnomality in enumerate(abnormalities):
                        scores[metric][abnomality] = filler
        
        return scores

def Find_metrics(scores: dict, y_true: np.ndarray, y_pred: np.ndarray):
        
        def _findProperScore(metric, y_true, y_pred):

                m = confusion_matrix(y_true, y_pred, labels=[0, 1])

                if metric == 'Accuracy':
                        return accuracy_score(y_true, y_pred)
                elif metric == 'Precision':
                        return precision_score(y_true, y_pred, zero_division=0)  
                elif metric == 'Recall':
                        return recall_score(y_true, y_pred, zero_division=0)
                elif metric == 'Specificity':
                        return m[0, 0] * 1.0 / (m[0, 0] + m[0, 1])
                else:
                        return f1_score(y_true, y_pred, zero_division=0)

        for jj, metric in enumerate(scores.keys()):
                for ii, cardio_class in enumerate(scores[metric].keys()):

                        scores[metric][cardio_class] = np.append(scores[metric][cardio_class], \
                                _findProperScore(metric, y_true[:,ii], y_pred[:,ii]))

        return scores

In [84]:
kf = KFold(n_splits=3, shuffle=True, random_state = 42)

m1_scores = Score_initator(filler = np.array([]))
m2_scores = Score_initator(filler = np.array([]))

#Initilaize the models that need to be trained
model_verbose = 0
model_2 = MyCNN(loss, opt, verbose = model_verbose)
'''model_3 = RF_Model(verbose = model_verbose)'''

#PCA initlization
#PCA_transformer = PCA_Transform(r = 10)

fold_count =1 
#TODO: there is no regularization in the layers so maybe can add that
for train_index, test_index in kf.split(X = tracings_test[:,1,1]):

        print('Fold #{}'.format(fold_count))

        X_train, X_test = tracings_test[train_index,:,:], tracings_test[test_index,:,:]
        y_train, y_test = labels_test[train_index], labels_test[test_index]

        #Put data in sequence for models 1 and 2 (CNN)
        train_seq, val_seq = ECGSequence.get_train_and_val(X_train, y_train, batch_size=64)
        test_seq = ECGSequence(X_test, y_test, batch_size=32)

        #Transform data with PCA for models 3 and 4
        '''for ii in range(X_train.shape[-1]):
                PCA_transformer.FitData(X_train[:,:,ii])
                PCA_X_train_temp, PCA_X_test_temp = PCA_transformer.\
                        TransformData(X_train[:,:,ii], X_test[:,:,ii])
                if ii == 0:
                        PCA_X_train = PCA_X_train_temp
                        PCA_X_test = PCA_X_test_temp
                else:
                        PCA_X_train = np.append(PCA_X_train, PCA_X_train_temp, axis = 1)
                        PCA_X_test = np.append(PCA_X_test, PCA_X_test_temp, axis = 1)'''

        #Train models
        model_2.train(train_seq, val_seq=val_seq)
        '''model_3.train(X = PCA_X_train, y = y_train)'''

        #Predict with the models - need to find optimal thresholds first for this test set
        m1_pred = ProcessOutputs(model_1.predict(test_seq, verbose=model_verbose), optimal_thresholds)
        m2_pred = ProcessOutputs(model_2.predict(test_seq), optimal_thresholds)
        #m3_pred = model_3.predict(PCA_X_train)

        #Find scores
        m1_scores = Find_metrics(m1_scores, y_test, m1_pred)
        m2_scores = Find_metrics(m2_scores, y_test, m2_pred)
        '''train_scores = Find_metrics(train_scores, 'm3', y_test, m3_pred)'''

        fold_count+=1


m1_scores_path = cwd + '\\MyOutputs\\m1_scores.pickle'
SaveObject(m1_scores, m1_scores_path)
m2_scores_path = cwd + '\\MyOutputs\\m2_scores.pickle'
SaveObject(m2_scores, m2_scores_path)



Fold #1
Fold #2
Fold #3


**Process the scores**

In [93]:
def ProcessScores(scores: dict, save_path: str):
    avg_scores = Score_initator(filler=dict())
    for sub_metric in avg_scores.keys():
        for cardiac_class in avg_scores[sub_metric].keys():
            sub_score = scores[sub_metric][cardiac_class]
            sub_score[sub_score == 0.0] = np.NaN
            avg_scores[sub_metric][cardiac_class] = {'Average': np.nanmean(sub_score),\
                'Var': np.nanvar(sub_score)}

    score_output = pd.DataFrame.from_dict({(i,j): avg_scores[i][j]
                           for i in avg_scores.keys() 
                           for j in avg_scores[i].keys()},
                       orient='index')

    score_output.to_csv(save_path)


m1_scores = LoadObject(m1_scores_path)
m2_scores = LoadObject(m2_scores_path)

ProcessScores(m1_scores, cwd + '\\MyOutputs\\m1_scores.csv')
ProcessScores(m2_scores, cwd + '\\MyOutputs\\m2_scores.csv')

C:\Users\Andrew Hart\AppData\Local\Temp\ipykernel_14920\2549717226.py:7: RuntimeWarning: Mean of empty slice
  avg_scores[sub_metric][cardiac_class] = {'Average': np.nanmean(sub_score),\
C:\Users\Andrew Hart\AppData\Local\Temp\ipykernel_14920\2549717226.py:8: RuntimeWarning: Degrees of freedom <= 0 for slice.
  'Var': np.nanvar(sub_score)}
